#Ch. 4 Exercises
<ol>
<li>[Dynamic 1D Robot Motion Model](#Dynamic_1d_robot_motion)</li>
<li>[Dynamic 1D Robot Posterior](#Dynamic_1d_robot_posterior)</li>
<li>[Dynamic 1D Robot Correlation](#Dynamic_1d_robot_correlation)</li>
<li>[Dynamic Bicycle Model](#Dynamic_Bicycle_Model)</li>

</ol>

## 1. Dynamic 1D Robot Motion Model <a id="Dynamic_1d_robot_motion"></a>

Develop a mathematical motion model that computes the posterior over the pose $x^{\prime}$ and the velocity $\dot{x}^{\prime}$ from an initial pose $x$ and velocity $\dot{x}$ assuming the acceleration \ddot{x} is the sum of a command and zero-mean gaussian noise with variance $\sigma^{2}$. Are $x^{\prime}$ and $\dot{x}^{\prime}$ correlated in the posterior?


This is the same probem as in earlier chapters. Assuming constant acceleration and no noise during the movement:

\begin{equation}
\dot{x}^{\prime}  = x^{\prime} - x \\
\ddot{x} = \dot{x}^{\prime} - \dot{x} \\
\end{equation}


$x^{\prime}$ and $\dot{x}^{\prime}$ are correlated as one can be derived from the other if $\mu$ is known exactly.


## 2. Dynamic 1D Robot Motion Posterior <a id="Dynamic_1d_robot_posterior"></a>

The probability of an acceleration given the control is $p(\ddot{x} \lvert u_t) = Probability(\ddot{x} - u_t, \sigma^{2})$

The posterior can be calculated by

\begin{equation}
p(\mu^{\prime}\ \lvert\ \mu, u) = \left\{\begin{array}{cc}
0, & \dot{x}^{\prime} \ne x^{\prime} - x \\
Probability((\dot{x}^{\prime} - \dot{x}) - u_t, \sigma^{2}), & else \end{array}\right\}
\end{equation}

It seems like this could be made more general by adding another acceleration at the end of the move, similar to the extra rotation at the end of the odemetry model's move, so that that most states wouldn't have a zero probability.





## 3. Dynamic 1D Robot Motion Correlation <a id="Dynamic_1d_robot_correlation"></a>

After controlling the robot with random accelerations for $T$ time intervals, for some large value of $T$ the final location $x$ and final velocity $\dot{x}$ will not be correlated. 

## 4. Dynamic Bicycle Model <a id="Dynamic_Bicycle_Model"></a>

Create a dynamics model for a bicycle using three state variables $x,y$ of the center of front tire and $\theta$ yaw of frame relative to an external reference, and controls forward velocity $v$ and steering angle $\alpha$ after time $\Delta t$.

Assuming noiseless control and movement and bicyle length $L$, the bike will move in a circle whose center is located at the intersection of the lines that run perpedicualr through the center of the tires.

The radius of the circle is 
\begin{equation}
r_{front} = \left|\frac{L}{sin(\alpha)}\right| \\
r_{back} = \left|\frac{L}{tan(\alpha)}\right|
\end{equation}

The center of the circle is at 
\begin{equation}
x_{c} = x - \frac{L}{sin(\alpha)} * sin(\theta + \alpha) \\
y_{c} = y + \frac{L}{sin(\alpha)} * cos(\theta + \alpha)
\end{equation}

The control $v$ is assumed to be the speed of the front tire. The distance traveled around the circumference of the circle $d = v * \Delta t$. The angle traversed during that time is $\Delta\theta = \frac{d}{r}$. The new position of the front tire will then be 

\begin{equation}
x^{\prime} = x_{c} + r_{front} * sin(\theta + \alpha + \Delta\theta) \\
y^{\prime} = y_{c} - r_{front} * cos(\theta + \alpha + \Delta\theta)
\end{equation}







# Bicycle Model

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
%pylab inline

class Bicycle:
    
    def __init__(self, length=1, x=0, y=0, theta=0, epsilon=0.001):
        self.length = length
        self.x = x
        self.y = y
        self.theta = theta
        self.epsilon = epsilon
        
    def _get_center(self, alpha):
        if np.abs(alpha) < self.epsilon:
            return None
        
        r = self.length / np.sin(alpha)
        xc = self.x - r * np.sin(self.theta+alpha)
        yc = self.y + r * np.cos(self.theta+alpha)
        
        return (xc, yc, r)
    
    def _get_new_pos(self, alpha, v, delta_t=1):
        if np.abs(alpha) < self.epsilon:
            new_x = self.x * v * np.cos(self.theta)
            new_y = self.y * v * np.sin(self.theta)
            new_theta = self.theta
        else:
            xc, yc, r = self._get_center(alpha)
            
            delta_theta = (v * delta_t) / r
            
            new_x = xc + r * sin(self.theta + alpha + delta_theta)
            new_y = yc - r * cos(self.theta + alpha + delta_theta)
            new_theta = self.theta + delta_theta
        return (new_x, new_y, new_theta)



# Debug Plots

In [ ]:
def plot_bicycle(b, alpha, ax, plot_opts='k'):
    # front tire
    ax.plot([x - b.length / 3.0 * cos(b.theta+alpha), x + b.length / 3.0 * cos(b.theta+alpha)],
            [y - b.length / 3.0 * sin(b.theta+alpha), y + b.length / 3.0 * sin(b.theta+alpha)],
            plot_opts)
    
    # back tire
    bx = x - b.length * cos(b.theta)
    by = y - b.length * sin(b.theta)
    ax.plot([bx - b.length / 3.0 * cos(b.theta), bx + b.length / 3.0 * cos(b.theta)],
            [by - b.length / 3.0 * sin(b.theta), by + b.length / 3.0 * sin(b.theta)],
            plot_opts)

thetas = np.array([0, 45, 90, -90, 180]) * np.pi / 180.0
alphas = np.array([-45, -30, 15, 30]) * np.pi / 180.0
velocities = np.linspace(1, 4, 10)

x,y = 0,0
L = 1
b = Bicycle(L, x, y)
for theta in thetas:
    fig = plt.figure(figsize=(12,4))
    for i, alpha in enumerate(alphas):
        b.theta = theta
        xc, yc, r = b._get_center(alpha)
        
        ax = fig.add_subplot(1,len(alphas),i+1, aspect='equal')
        plot_bicycle(b, alpha, ax)
        ax.plot(xc,yc,'yo')

        
        
        for v in velocities:        
            xn, yn, tn = b._get_new_pos(alpha, v)
            
            bound = 5
            ax.set_xlim((-bound, bound))
            ax.set_ylim((-bound, bound))
            ax.plot(xn,yn,'go')
        
